# "Stock" Grade Neural Network
### Starter Kernel by ``Magichanics`` 
*([GitHub](https://github.com/Magichanics) - [Kaggle](https://www.kaggle.com/magichanics))*

With more features from public kernels, as well as the idea of using Neural Networks for modelling, I've decided to do some experimenting myself in hopes of producing the best results. Feel free to post suggestions or criticisms!

## Table of Contents

* [Step 1. Merging Datasets](#section1)
* [Step 2. Feature Engineering](#section2)
* [Step 3. Modelling using Keras' Neural Network](#section3)
* [Step 4. Applying the Model](#section4)

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
from itertools import chain

import matplotlib.pyplot as plt

In [ ]:
# import environment for data
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()
market_train_df = market_train_df.tail(40_000)
news_train_df = news_train_df.tail(100_000)

<a id='section1'></a>
## Step 1. Merging Datasets

While most of the notebooks focuses only on the market dataset, I'm going to attempt on bringing both the news and market dataset together.

In [ ]:
market_train_df.head()

In [ ]:
news_train_df.head()

### Cleaning Data
Some of the rows in the dataset does not make sense, so we will be removing said rows.

In [ ]:
def clean_data(market_df, news_df):
    
    # get rid of blank headlines
    news_df = news_df[news_df.headline != '']
    
    return market_df, news_df

In [ ]:
market_train_df, news_train_df = clean_data(market_train_df, news_train_df)

### Expanding News data
We are going to be splitting the news data by assetCode.

In [ ]:
def expanding_news(news_df):
    
    # split to list
    news_output = news_df.copy()
    news_output['assetCodes'] = news_output['assetCodes'].str.findall(f"'([\w\./]+)'")
    
    # separate to assetcodes
    assetCodes_expanded = list(chain(*news_output['assetCodes']))
    assetCodes_index = news_df.index.repeat(news_output['assetCodes'].apply(len))
    assert len(assetCodes_index) == len(assetCodes_expanded)
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})
    
    # merge to dataframe
    merging_cols = [f for f in news_output if f not in ['assetCodes', 'sourceId']]
    news_df_expanded = pd.merge(df_assetCodes, news_output[merging_cols], left_on='level_0', 
                                right_index=True, suffixes=(['','_old']))
    
    return news_df_expanded

In [ ]:
temp_expand_df = expanding_news(news_train_df)

In [ ]:
temp_expand_df.tail()

### Merge by Closest Time to News Article
We will be adding each news article to their closest market row in terms of time, but with no news article being over one hour.

<a id='section2'></a>
## Step 2. Feature Engineering

From Quant features to text processing features.

### News Features
* Last News Article - This feature will have the number of days it has been since a news article has targeted the given assetCode
* Number of Articles Today/Week/Month - Fetches the number of Articles that was written on the assetCode during the given timeframe.

### Entire Market and Individual Asset Quant Features
We are going to be obtaining Quant Features from both the entire market dataframe and from each individual asset based on assetCode.

### Text Processing with CountVectorizer and TfidfVectorizer
We are going to be using CountVectorizer and TfidfVectorizer on the headlines to determine its influence on the target column.

### Clustering
We will be clustering the open and close features using KMeans.

In [ ]:
def clustering(X):

    def cluster_modelling(features):
        df_set = X[features]
        cluster_model = KMeans(n_clusters = 8)
        cluster_model.fit(df_set)
        return cluster_model.predict(df_set)
    
    # get columns:
    vol_cols = [f for f in X.columns if f != 'volume' and 'volume' in f]
    novelty_cols = [f for f in X.columns if 'novelty' in f]
    
    # fill nulls
    cluster_cols = novelty_cols + vol_cols + ['open', 'close']
    X[cluster_cols] = X[cluster_cols].fillna(0)
    
    X['cluster_open_close'] = cluster_modelling(['open', 'close'])
    X['cluster_volume'] = cluster_modelling(vol_cols)
    X['cluster_novelty'] = cluster_modelling(novelty_cols)
    
    return df

<a id='section3'></a>
## Step 3. Modelling using Keras' Neural Network

### Preparing Datasets for Modelling
We will convert all the numerical and categorical datasets into rows that the neural network can process.

In [ ]:
from sklearn.preprocessing import StandardScaler

# scale numerical columns
scaler = StandardScaler()

X_train = scaler.fit_transform(market_train_df[test_cols].fillna(0))

y_train = market_train_df['returnsOpenNextMktres10']

In [ ]:
def get_cols(X_train):
    
    # get numerical and categorical columns
    num_cols = [f for f in X_train.columns if X_train[f].dtype == 'int' or X_train[f].dtype == 'float' and f not in ['universe', 'returnsOpenNextMktres10']]
    cat_cols = [f for f in X_train.columns if f not in num_cols and f not in ['universe', 'returnsOpenNextMktres10']]
    
    return num_cols, cat_cols

### Fixed Training Split
The reason why we need to do a fixed training test split that fetches the last few rows of the training dataset is to avoid odd results, since randomly choosing rows will cause the validation dataset to be filled with rows with different timestamps.

In [ ]:
def fixed_train_test_split(X, y, train_size):
    
    # round train size
    train_size = int(train_size * len(X))
    
    # split data
    X_train, y_train = X[train_size:], y[train_size:]
    X_valid, y_valid = X[:train_size], y[:train_size]
    
    return X_train, y_train, X_valid, y_valid

In [ ]:
X_train, y_train, X_valid, y_valid = fixed_train_test_split(X_train, y_train, 5000)

In [ ]:
# original from https://www.kaggle.com/christofhenkel/market-data-nn-baseline
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, BatchNormalization
from keras.losses import binary_crossentropy

# categorical data
categorical_inputs = []
for cat in cat_cols:
    categorical_inputs.append(Input(shape=[1], name=cat))

categorical_embeddings = []
for i, cat in enumerate(cat_cols):
    categorical_embeddings.append(Embedding(embed_sizes[i], 10)(categorical_inputs[i]))
    
categorical_logits = Flatten()(categorical_embeddings[0])
categorical_logits = Dense(32,activation='relu')(categorical_logits)

# numerical data
numerical_inputs = Input(shape=(11,), name='num')
numerical_logits = numerical_inputs
numerical_logits = BatchNormalization()(numerical_logits)

numerical_logits = Dense(128,activation='relu')(numerical_logits)
numerical_logits = Dense(64,activation='relu')(numerical_logits)

# combined
logits = Concatenate()([numerical_logits,categorical_logits])
logits = Dense(128,activation='relu')(logits)
logits = Dense(64,activation='relu')(logits)
out = Dense(1, activation='sigmoid')(logits)

model = Model(inputs = categorical_inputs + [numerical_inputs], outputs=out)
model.compile(optimizer='adam',loss=binary_crossentropy)

In [ ]:
get_cols(market_train_df)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# set cylical learning rate per epoch
learning_rate = 1e-4
dynamic_lr = LearningRateScheduler(lambda epoch: learning_rate * 0.99 ** epoch)

# set early stopping
early_stop = EarlyStopping(patience=3)

model.fit(X_train,y_train.astype(int),
          validation_data=(X_valid,y_valid.astype(int)),
          epochs=200,
          verbose=0,
         callbacks=[dynamic_lr, early_stop]) 

In [ ]:
model.predict(X_valid)

<a id='section4'></a>
## Step 4. Applying the Model


**Sources:**
* [Market Data NN Baseline by Christofhenkel](https://www.kaggle.com/christofhenkel/market-data-nn-baseline)
* [a simple model using the market and news data by Bguberfain](https://www.kaggle.com/bguberfain/a-simple-model-using-the-market-and-news-data)
* [Amateur Hour - Using Headlines to Predict Stocks by Magichanics](https://www.kaggle.com/magichanics/amateur-hour-using-headlines-to-predict-stocks)
* [Simple Quant Features by Youhanlee](https://www.kaggle.com/youhanlee/simple-quant-features-using-python)